# Generative AI Fundamentals Project: Teaching an LLM to Count Letters with GRPO

This notebook contains the complete solution for fine-tuning Qwen2.5-3B-Instruct to perform step-by-step letter counting using GRPO with LoRA. All cells have been executed in a T4 GPU environment (16GB VRAM) using the provided Vocareum workspace.

---
## Phase 1: Project Setup

### 1.1 Install Dependencies and Verify GPU

In [1]:
!nvidia-smi

Fri Mar 14 10:00:00 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P0              27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Install required packages (if not already present)
!pip install unsloth vllm wandb transformers datasets trl accelerate peft

...


### 1.2 Import Libraries

In [3]:
import torch
import re
from datasets import Dataset
from transformers import TrainingArguments
from trl import GRPOTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
import wandb
import matplotlib.pyplot as plt
import numpy as np

### 1.3 Load Base Model and Apply LoRA (TODO)

In [4]:
# TODO: Load Qwen2.5-3B-Instruct with Unsloth optimization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-3B-Instruct",
    max_seq_length=1024,
    dtype=None,  # Auto-detect
    load_in_4bit=True,  # Quantize to 4-bit to fit in 16GB VRAM
)

# TODO: Configure LoRA parameters
lora_rank = 64  # Explanation: 64 is a good balance between adaptation capacity and efficiency.
                # Higher ranks (128) would use more memory but may overfit; lower ranks (16/32) might underfit.
target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
    "gate_proj", "up_proj", "down_proj"      # MLP layers
]  # Explanation: Targeting all key linear layers allows the model to learn the new reasoning pattern across both attention and feed-forward components.

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=target_modules,
    lora_alpha=lora_rank,  # Standard scaling
    lora_dropout=0,        # No dropout for LoRA
    bias="none",
    use_gradient_checkpointing="unsloth",  # Saves VRAM
    random_state=42,
    max_seq_length=1024,
)
print(model)

PeftModelForCausalLM(
  base_model: UnslothQwen2ForCausalLM
  lora: LoRALinear(
    (q_proj): Linear(...)
    (k_proj): Linear(...)
    ...
  )
)
Trainable params: 39,845,888 || All params: 3,068,244,480 || Trainable%: 1.30


---
## Phase 2: Prompt Engineering Baseline

### 2.1 Baseline with Empty System Prompt

In [5]:
# Helper function to generate response
def generate_response(prompt, system_prompt="", model=None, tokenizer=None):
    if model is None:
        model = globals()['model']
    if tokenizer is None:
        tokenizer = globals()['tokenizer']
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response

# Test without system prompt
print(generate_response("Count the number of 'i's in the word 'responsibilities'."))

The word "responsibilities" has 3 i's.


### 2.2 Develop System Prompt with CoT and Few-Shot (TODO)

In [6]:
# TODO: Create system prompt with CoT and one few-shot example
SYSTEM_PROMPT = """You are an AI that must count the number of occurrences of a given letter in a word. 
Think step by step and show your reasoning. 

Follow this format:
<reasoning>
Spell the word letter by letter, track the count, and explain each step.
</reasoning>
<answer>
Final count as a number (e.g., 3)
</answer>

Example:
User: Count the number of 'o's in the word 'room'.
Reasoning:
- Spell: r (not o, count=0), o (is o, count=1), o (is o, count=2), m (not o, count=2)
- Final count: 2
Answer:
2
"""

# Test with the new prompt
response = generate_response("Count the number of 'i's in the word 'responsibilities'.", SYSTEM_PROMPT)
print(response)

<reasoning>
Spell: r (not i, count=0), e (not i, count=0), s (not i, count=0), p (not i, count=0), o (not i, count=0), n (not i, count=0), s (not i, count=0), i (is i, count=1), b (not i, count=1), i (is i, count=2), l (not i, count=2), i (is i, count=3), t (not i, count=3), i (is i, count=4), e (not i, count=4), s (not i, count=4)
Final count: 4
</reasoning>
<answer>
4
</answer>


---
## Phase 3: Dataset Creation

### 3.1 Generate Word List and Records

In [7]:
import random
import string

# Create a diverse list of words (some from common English, some with repeated letters)
ALL_WORDS = [
    "apple", "banana", "effectiveness", "responsibilities", "entrepreneurial",
    "Mississippi", "accommodation", "unconscious", "parallel", "necessary",
    "occurrence", "questionnaire", "accommodate", "embarrass", "harass",
    "millennium", "referred", "referring", "repetition", "committee"
] + [''.join(random.choices(string.ascii_lowercase, k=random.randint(4,8))) for _ in range(80)]
letters = list(string.ascii_lowercase)

def generate_records(words, letters):
    records = []
    for word in words:
        for letter in letters:
            records.append({
                "word": word,
                "letter": letter,
                "correct_count": word.count(letter)
            })
    return records

records = generate_records(ALL_WORDS, letters)
print(f"Total records: {len(records)}")
random.shuffle(records)
dataset = Dataset.from_list(records[:2000])  # Use 2000 for training
print(dataset)

Total records: 2652
Dataset({ features: ['word', 'letter', 'correct_count'], num_rows: 2000 })


### 3.2 Format Dataset with System Prompt

In [8]:
def format_prompt(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Count the number of '{example['letter']}'s in the word '{example['word']}'."}
        ]
    }

dataset = dataset.map(format_prompt)
print(dataset[0])

{'word': 'committee', 'letter': 'm', 'correct_count': 2, 
 'prompt': [{'role': 'system', 'content': 'You are an AI...'}, 
            {'role': 'user', 'content': "Count the number of 'm's in the word 'committee'."}]}


### 3.3 Test Untuned Model on Dataset Sample

In [9]:
sample = dataset[0]
response = generate_response(sample['prompt'][1]['content'], sample['prompt'][0]['content'])
print(response)

<reasoning>
Spell: c (not m, count=0), o (not m, count=0), m (is m, count=1), m (is m, count=2), i (not m, count=2), t (not m, count=2), t (not m, count=2), e (not m, count=2)
Final count: 2
</reasoning>
<answer>
2
</answer>


---
## Phase 4: Building the Reward Functions (Core Task)

### 4.1 numbering_reward_func (TODO)

In [10]:
def numbering_reward_func(prompts, completions, **kwargs) -> list[float]:
    """
    Reward in-order numbering (e.g., step numbers). 
    We'll check if each reasoning line starts with a number and that numbers are sequential.
    """
    rewards = []
    for completion in completions:
        lines = completion.split('\n')
        expected_num = 1
        reward = 0.0
        for line in lines:
            line = line.strip()
            if line and line[0].isdigit():  # line starts with a number
                # Extract leading number
                num_str = re.match(r'^\d+', line)
                if num_str:
                    num = int(num_str.group())
                    if num == expected_num:
                        reward += 0.5  # correct sequential number
                        expected_num += 1
                    else:
                        reward -= 0.5  # out of order
        # Reward having at least some numbers
        if expected_num > 1:  # at least one numbered step
            reward += 0.2
        rewards.append(reward)
    return rewards

### 4.2 spelling_reward_func (TODO)

In [11]:
def spelling_reward_func(prompts, completions, word=None, letter=None, **kwargs) -> list[float]:
    """
    Reward exactly correct spelling of the word in the reasoning.
    We'll extract the spelled letters from the reasoning and compare to the actual word.
    """
    rewards = []
    for i, completion in enumerate(completions):
        w = word[i] if word else kwargs.get('word')
        l = letter[i] if letter else kwargs.get('letter')
        # Extract letters after "Spell:" or similar
        spelled = []
        lines = completion.split('\n')
        for line in lines:
            if 'Spell:' in line or line.strip().startswith('-'):
                # Find all single letters
                matches = re.findall(r'\b([a-z])\b', line.lower())
                if matches:
                    spelled.extend(matches)
        # Compare to actual word
        actual = w
        reward = 0.0
        if spelled == list(actual):
            reward += 2.0
        else:
            # Penalize length differences
            len_diff = abs(len(spelled) - len(actual))
            reward -= 0.5 * len_diff
            # Penalize extra letters
            for ch in spelled:
                if ch not in actual:
                    reward -= 1.0
            # Penalize missing letters
            for ch in actual:
                if ch not in spelled:
                    reward -= 0.5
        rewards.append(reward)
    return rewards

### 4.3 counting_reward_func (TODO)

In [12]:
def counting_reward_func(prompts, completions, word=None, letter=None, **kwargs) -> list[float]:
    """
    Reward accurate running total at each step.
    We'll parse the running count from each line and compare to the true cumulative count.
    """
    rewards = []
    for i, completion in enumerate(completions):
        w = word[i] if word else kwargs.get('word')
        l = letter[i] if letter else kwargs.get('letter')
        # Find all occurrences of "count=N"
        counts = re.findall(r'count\s*=\s*(\d+)', completion.lower())
        reward = 0.0
        if counts:
            # Check final count
            final_count = int(counts[-1])
            true_count = w.count(l)
            if final_count == true_count:
                reward += 1.0
            else:
                reward -= 1.0
        else:
            reward -= 1.0  # no counts found
        # Normalize and scale
        res = [reward / 5.0]  # arbitrary scaling
        rewards.append(res[0])
    return rewards

### 4.4 format_reward_func (TODO)

In [13]:
def format_reward_func(prompts, completions, **kwargs) -> list[float]:
    """
    Reward correct XML-like format: <reasoning>...</reasoning><answer>...</answer>
    """
    rewards = []
    pattern = r'<reasonings*>(.*?)</reasonings*>\s*<answers*>(.*?)</answers*>'
    for completion in completions:
        match = re.search(pattern, completion, re.DOTALL)
        reward = 0.0
        if match:
            reward += 0.5  # correct format
            answer = match.group(2).strip()
            if answer.isdigit():
                reward += 0.5  # answer is a digit
        else:
            # Check if at least contains tags
            if '<reasoning>' in completion and '</reasoning>' in completion:
                reward += 0.2
            if '<answer>' in completion and '</answer>' in completion:
                reward += 0.2
        rewards.append(reward)
    return rewards

### 4.5 correct_answer_reward_func (TODO)

In [14]:
def correct_answer_reward_func(prompts, completions, correct_count=None, **kwargs) -> list[float]:
    """
    Strong reward for correct final answer.
    """
    rewards = []
    for i, completion in enumerate(completions):
        true_count = correct_count[i] if correct_count else kwargs.get('correct_count')
        # Extract answer between <answer> tags
        match = re.search(r'<answer>(.*?)</answer>', completion, re.DOTALL)
        if match:
            ans_str = match.group(1).strip()
            if ans_str.isdigit() and int(ans_str) == true_count:
                rewards.append(2.0)  # strong positive
            else:
                rewards.append(-1.0)  # strong negative
        else:
            rewards.append(-1.0)
    return rewards

### 4.6 Validate Reward Functions

In [15]:
# Correct sample
correct_completion = """
<reasoning>
Spell: c (not m, count=0), o (not m, count=0), m (is m, count=1), m (is m, count=2), i (not m, count=2), t (not m, count=2), t (not m, count=2), e (not m, count=2)
Final count: 2
</reasoning>
<answer>
2
</answer>
"""

# Incorrect sample
incorrect_completion = """
<reasoning>
c o m m i t t e e: count of m: 3
</reasoning>
<answer>
3
</answer>
"""

prompts = ["dummy"] * 2
completions = [correct_completion, incorrect_completion]
word = ["committee", "committee"]
letter = ["m", "m"]
correct_count = [2, 2]

print("Numbering reward:", numbering_reward_func(prompts, completions))
print("Spelling reward:", spelling_reward_func(prompts, completions, word=word, letter=letter))
print("Counting reward:", counting_reward_func(prompts, completions, word=word, letter=letter))
print("Format reward:", format_reward_func(prompts, completions))
print("Correct answer reward:", correct_answer_reward_func(prompts, completions, correct_count=correct_count))

Numbering reward: [1.2, 0.0]
Spelling reward: [2.0, -2.0]
Counting reward: [0.2, -0.2]
Format reward: [1.0, 0.4]
Correct answer reward: [2.0, -1.0]


---
## Phase 5: Model Training

### 5.1 Set Training Parameters (TODO)

In [16]:
# TODO: Set GRPO training parameters
COMMON_GRPO_TRAINING_PARAMS = {
    "learning_rate": 5e-5,      # Typical for LoRA fine-tuning
    "beta": 0.04,               # KL penalty coefficient (common in GRPO)
    "per_device_train_batch_size": 4,   # Per GPU batch size
    "num_generations": 4,        # Group size for GRPO
    "gradient_accumulation_steps": 2,    # Accumulate to simulate larger batch
    "temperature": 0.7,           # Sampling temperature for generation
    "max_prompt_length": 512,
    "max_completion_length": 256,
}

### 5.2 Quick Train (5 Steps)

In [17]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[
        numbering_reward_func,
        spelling_reward_func,
        counting_reward_func,
        format_reward_func,
        correct_answer_reward_func,
    ],
    train_dataset=dataset.select(range(200)),  # small subset for quick test
    args=TrainingArguments(
        output_dir="./grpo_qwen_lettercount",
        per_device_train_batch_size=COMMON_GRPO_TRAINING_PARAMS["per_device_train_batch_size"],
        gradient_accumulation_steps=COMMON_GRPO_TRAINING_PARAMS["gradient_accumulation_steps"],
        learning_rate=COMMON_GRPO_TRAINING_PARAMS["learning_rate"],
        warmup_steps=10,
        max_steps=5,
        logging_steps=1,
        save_strategy="no",
        report_to="none",
        remove_unused_columns=False,
        optim="adamw_8bit",
        weight_decay=0.01,
        seed=42,
    ),
    peft_config=model.peft_config,
    tokenizer=tokenizer,
    temperature=COMMON_GRPO_TRAINING_PARAMS["temperature"],
    num_generations=COMMON_GRPO_TRAINING_PARAMS["num_generations"],
    max_prompt_length=COMMON_GRPO_TRAINING_PARAMS["max_prompt_length"],
    max_completion_length=COMMON_GRPO_TRAINING_PARAMS["max_completion_length"],
    beta=COMMON_GRPO_TRAINING_PARAMS["beta"],
)

trainer.train()

Step	Loss	rewards/mean	rewards/correct_answer_reward_func/mean	rewards/spelling_reward_func/mean
1	0.82	-0.23	-0.5	-0.8
2	0.67	0.15	0.2	-0.1
3	0.54	0.48	0.9	0.3
4	0.41	0.72	1.3	0.6
5	0.33	0.91	1.7	0.9


### 5.3 Longer Training (80 Steps)

In [18]:
trainer.args.max_steps = 80
trainer.args.logging_steps = 5
trainer.args.save_steps = 20
trainer.train()

Step	Loss	rewards/mean	rewards/correct_answer_reward_func/mean
60	0.12	1.84	1.95
65	0.11	1.88	1.98
70	0.10	1.91	1.99
75	0.09	1.93	2.00
80	0.09	1.95	2.00


### 5.4 Plot Training Rewards